# Création d'un second réseau de neurone

Cette partie du code présente la façon dont nous avons construit notre second réseau de neurone, prenant en paramètre la moyenne des buts marqués par match avant que le match se joue, la moyenne de l'expected goal (xg) par match avant que le match se joue, et la moyenne des tirs cadrés par match avant que le match se joue.

Le réseau de neurone suivant la méthode préconisée par François Chollet dans l'ouvrage 'L'apprentissage profond avec Python', en prenant en considération toutes les difficultés liées à l'utilisation d'un dataset avec peu de lignes (1500).

## Création du dataset pour la saison 2021

In [ ]:
import pandas as pd

# Charger le dataset
file_path = 'matches.csv'  # Remplacer par le chemin correct vers ton dataset
data = pd.read_csv(file_path)

# Filtrer pour garder seulement la saison 2021
data_2021 = data[data['season'] == 2021].copy()

# Supprimer les colonnes non souhaitées
columns_to_remove = ['Unnamed: 0', 'date', 'time', 'comp', 'day', 'attendance', 'captain', 'formation',
                     'referee', 'match report', 'notes', 'dist', 'fk', 'pk', 'pkatt']
data_2021.drop(columns=columns_to_remove, inplace=True)

# Convertir la colonne 'round' en un numéro de semaine
data_2021['week_number'] = data_2021['round'].apply(lambda x: int(x.split(' ')[1]))

# Trier par équipe et numéro de semaine pour réinitialiser les calculs correctement
data_2021.sort_values(by=['team', 'week_number'], inplace=True)

# Initialiser les colonnes pour les cumuls et moyennes
data_2021['som_goals'] = 0
data_2021['moy_goals'] = 0
data_2021['som_goals_pris'] = 0
data_2021['moy_goals_pris'] = 0
data_2021['som_xg'] = 0
data_2021['moy_xg'] = 0
data_2021['som_xga'] = 0
data_2021['moy_xga'] = 0
data_2021['som_poss'] = 0
data_2021['moy_poss'] = 0
data_2021['som_sot'] = 0
data_2021['moy_sot'] = 0
data_2021['total_points'] = 0
data_2021['DB'] = 0

# Calculer les valeurs pour chaque équipe indépendamment
teams = data_2021['team'].unique()

for team in teams:
    team_data = data_2021[data_2021['team'] == team].copy()
    team_data = team_data.sort_values(by=['week_number'])

    team_data['som_goals'] = team_data['gf'].cumsum().shift().fillna(0)
    team_data['moy_goals'] = team_data['som_goals'] / (team_data['week_number'] - 1)
    team_data['moy_goals'] = team_data['moy_goals'].replace([float('inf'), -float('inf')], 0)

    team_data['som_goals_pris'] = team_data['ga'].cumsum().shift().fillna(0)
    team_data['moy_goals_pris'] = team_data['som_goals_pris'] / (team_data['week_number'] - 1)
    team_data['moy_goals_pris'] = team_data['moy_goals_pris'].replace([float('inf'), -float('inf')], 0)

    team_data['som_xg'] = team_data['xg'].cumsum().shift().fillna(0)
    team_data['moy_xg'] = team_data['som_xg'] / (team_data['week_number'] - 1)
    team_data['moy_xg'] = team_data['moy_xg'].replace([float('inf'), -float('inf')], 0)

    team_data['som_xga'] = team_data['xga'].cumsum().shift().fillna(0)
    team_data['moy_xga'] = team_data['som_xga'] / (team_data['week_number'] - 1)
    team_data['moy_xga'] = team_data['moy_xga'].replace([float('inf'), -float('inf')], 0)

    team_data['som_poss'] = team_data['poss'].cumsum().shift().fillna(0)
    team_data['moy_poss'] = team_data['som_poss'] / (team_data['week_number'] - 1)
    team_data['moy_poss'] = team_data['moy_poss'].replace([float('inf'), -float('inf')], 0)

    team_data['som_sot'] = team_data['sot'].cumsum().shift().fillna(0)
    team_data['moy_sot'] = team_data['som_sot'] / (team_data['week_number'] - 1)
    team_data['moy_sot'] = team_data['moy_sot'].replace([float('inf'), -float('inf')], 0)

    team_data['match_points'] = team_data['result'].map({'W': 3, 'D': 1, 'L': 0})
    team_data['total_points'] = team_data['match_points'].cumsum().shift().fillna(0)

    team_data['DB'] = team_data['som_goals'] - team_data['som_goals_pris']

    data_2021.loc[team_data.index, 'som_goals'] = team_data['som_goals']
    data_2021.loc[team_data.index, 'moy_goals'] = team_data['moy_goals']
    data_2021.loc[team_data.index, 'som_goals_pris'] = team_data['som_goals_pris']
    data_2021.loc[team_data.index, 'moy_goals_pris'] = team_data['moy_goals_pris']
    data_2021.loc[team_data.index, 'som_xg'] = team_data['som_xg']
    data_2021.loc[team_data.index, 'moy_xg'] = team_data['moy_xg']
    data_2021.loc[team_data.index, 'som_xga'] = team_data['som_xga']
    data_2021.loc[team_data.index, 'moy_xga'] = team_data['moy_xga']
    data_2021.loc[team_data.index, 'som_poss'] = team_data['som_poss']
    data_2021.loc[team_data.index, 'moy_poss'] = team_data['moy_poss']
    data_2021.loc[team_data.index, 'som_sot'] = team_data['som_sot']
    data_2021.loc[team_data.index, 'moy_sot'] = team_data['moy_sot']
    data_2021.loc[team_data.index, 'total_points'] = team_data['total_points']
    data_2021.loc[team_data.index, 'DB'] = team_data['DB']

# Calculer le rang après avoir réinitialisé les valeurs
data_2021['rank'] = 0
unique_weeks = data_2021['week_number'].unique()
for week in unique_weeks:
    weekly_data = data_2021[data_2021['week_number'] == week].copy()
    weekly_data = weekly_data.sort_values(by=['total_points', 'DB'], ascending=[False, False])
    weekly_data['rank'] = range(1, len(weekly_data) + 1)
    data_2021.loc[weekly_data.index, 'rank'] = weekly_data['rank']

# Sauvegarder le dataset modifié
output_file_path = 'parfait_database(1).csv'  # Remplacer par le chemin souhaité
data_2021.to_csv(output_file_path, index=False)


# Création du dataset pour la saison 2022

In [ ]:
import pandas as pd

# Charger le dataset
file_path = 'matches.csv'  # Remplacer par le chemin correct vers ton dataset
data = pd.read_csv(file_path)

# Filtrer pour garder seulement la saison 2021
data_2021 = data[data['season'] == 2022].copy()

# Supprimer les colonnes non souhaitées
columns_to_remove = ['Unnamed: 0', 'date', 'time', 'comp', 'day', 'attendance', 'captain', 'formation',
                     'referee', 'match report', 'notes', 'dist', 'fk', 'pk', 'pkatt']
data_2021.drop(columns=columns_to_remove, inplace=True)

# Convertir la colonne 'round' en un numéro de semaine
data_2021['week_number'] = data_2021['round'].apply(lambda x: int(x.split(' ')[1]))

# Trier par équipe et numéro de semaine pour réinitialiser les calculs correctement
data_2021.sort_values(by=['team', 'week_number'], inplace=True)

# Initialiser les colonnes pour les cumuls et moyennes
data_2021['som_goals'] = 0
data_2021['moy_goals'] = 0
data_2021['som_goals_pris'] = 0
data_2021['moy_goals_pris'] = 0
data_2021['som_xg'] = 0
data_2021['moy_xg'] = 0
data_2021['som_xga'] = 0
data_2021['moy_xga'] = 0
data_2021['som_poss'] = 0
data_2021['moy_poss'] = 0
data_2021['som_sot'] = 0
data_2021['moy_sot'] = 0
data_2021['total_points'] = 0
data_2021['DB'] = 0

# Calculer les valeurs pour chaque équipe indépendamment
teams = data_2021['team'].unique()

for team in teams:
    team_data = data_2021[data_2021['team'] == team].copy()
    team_data = team_data.sort_values(by=['week_number'])

    team_data['som_goals'] = team_data['gf'].cumsum().shift().fillna(0)
    team_data['moy_goals'] = team_data['som_goals'] / (team_data['week_number'] - 1)
    team_data['moy_goals'] = team_data['moy_goals'].replace([float('inf'), -float('inf')], 0)

    team_data['som_goals_pris'] = team_data['ga'].cumsum().shift().fillna(0)
    team_data['moy_goals_pris'] = team_data['som_goals_pris'] / (team_data['week_number'] - 1)
    team_data['moy_goals_pris'] = team_data['moy_goals_pris'].replace([float('inf'), -float('inf')], 0)

    team_data['som_xg'] = team_data['xg'].cumsum().shift().fillna(0)
    team_data['moy_xg'] = team_data['som_xg'] / (team_data['week_number'] - 1)
    team_data['moy_xg'] = team_data['moy_xg'].replace([float('inf'), -float('inf')], 0)

    team_data['som_xga'] = team_data['xga'].cumsum().shift().fillna(0)
    team_data['moy_xga'] = team_data['som_xga'] / (team_data['week_number'] - 1)
    team_data['moy_xga'] = team_data['moy_xga'].replace([float('inf'), -float('inf')], 0)

    team_data['som_poss'] = team_data['poss'].cumsum().shift().fillna(0)
    team_data['moy_poss'] = team_data['som_poss'] / (team_data['week_number'] - 1)
    team_data['moy_poss'] = team_data['moy_poss'].replace([float('inf'), -float('inf')], 0)

    team_data['som_sot'] = team_data['sot'].cumsum().shift().fillna(0)
    team_data['moy_sot'] = team_data['som_sot'] / (team_data['week_number'] - 1)
    team_data['moy_sot'] = team_data['moy_sot'].replace([float('inf'), -float('inf')], 0)

    team_data['match_points'] = team_data['result'].map({'W': 3, 'D': 1, 'L': 0})
    team_data['total_points'] = team_data['match_points'].cumsum().shift().fillna(0)

    team_data['DB'] = team_data['som_goals'] - team_data['som_goals_pris']

    data_2021.loc[team_data.index, 'som_goals'] = team_data['som_goals']
    data_2021.loc[team_data.index, 'moy_goals'] = team_data['moy_goals']
    data_2021.loc[team_data.index, 'som_goals_pris'] = team_data['som_goals_pris']
    data_2021.loc[team_data.index, 'moy_goals_pris'] = team_data['moy_goals_pris']
    data_2021.loc[team_data.index, 'som_xg'] = team_data['som_xg']
    data_2021.loc[team_data.index, 'moy_xg'] = team_data['moy_xg']
    data_2021.loc[team_data.index, 'som_xga'] = team_data['som_xga']
    data_2021.loc[team_data.index, 'moy_xga'] = team_data['moy_xga']
    data_2021.loc[team_data.index, 'som_poss'] = team_data['som_poss']
    data_2021.loc[team_data.index, 'moy_poss'] = team_data['moy_poss']
    data_2021.loc[team_data.index, 'som_sot'] = team_data['som_sot']
    data_2021.loc[team_data.index, 'moy_sot'] = team_data['moy_sot']
    data_2021.loc[team_data.index, 'total_points'] = team_data['total_points']
    data_2021.loc[team_data.index, 'DB'] = team_data['DB']

# Calculer le rang après avoir réinitialisé les valeurs
data_2021['rank'] = 0
unique_weeks = data_2021['week_number'].unique()
for week in unique_weeks:
    weekly_data = data_2021[data_2021['week_number'] == week].copy()
    weekly_data = weekly_data.sort_values(by=['total_points', 'DB'], ascending=[False, False])
    weekly_data['rank'] = range(1, len(weekly_data) + 1)
    data_2021.loc[weekly_data.index, 'rank'] = weekly_data['rank']

# Sauvegarder le dataset modifié
output_file_path = 'parfait_database(2).csv'  # Remplacer par le chemin souhaité
data_2021.to_csv(output_file_path, index=False)


# Concaténation des deux datasets

In [ ]:
import pandas as pd

# Charger les deux datasets
dataset_2021_path = 'parfait_database(1).csv'
dataset_2022_path = 'parfait_database(2).csv'

dataset_2021 = pd.read_csv(dataset_2021_path)
dataset_2022 = pd.read_csv(dataset_2022_path)

# Concatenation des deux datasets
combined_dataset = pd.concat([dataset_2021, dataset_2022], ignore_index=True)

# Sauvegarder le dataset combiné
output_combined_path = 'combined_dataset_2021_2022.csv'
combined_dataset.to_csv(output_combined_path, index=False)


# 1) Premier code pour obtenir un résultat en se basant sur le cours et la méthode d'optimisation ADAM

In [ ]:
#Code cours



import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Charger les données
data = pd.read_csv('combined_dataset_2021_2022.csv')

# Prétraitement des données
data['venue'] = data['venue'].apply(lambda x: 1 if x == 'Home' else 0)
data.fillna(0, inplace=True)  # Remplacer les NaN par 0 pour la première journée

# Sélectionner les colonnes pertinentes
features = ['venue','moy_goals', 'moy_xg', 'moy_sot']
target = 'result'

# Encoder le résultat
label_encoder = LabelEncoder()
data['result_encoded'] = label_encoder.fit_transform(data[target])

# Séparer les features et le target
X = data[features]
y = data['result_encoded']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normaliser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Construire le modèle
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes pour W, D, L
])

# Compiler le modèle
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test accuracy: {accuracy * 100:.2f}%')


## 1.1 Nettoyage de données : Code cours actualisé avec la première ligne qu'on ne prend pas en compte car ce n'est pas representatif

## 1.2 On a normalisé les données à l'aide de StandarScaler()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Charger les données
data = pd.read_csv('combined_dataset_2021_2022.csv')

# Exclure les matchs de la première semaine
data = data[data['round'] != 'Matchweek 1']

# Prétraitement des données
data['venue'] = data['venue'].apply(lambda x: 1 if x == 'Home' else 0)

# Sélectionner les colonnes pertinentes
features = ['venue','moy_goals', 'moy_xg', 'moy_sot']
target = 'result'

# Encoder le résultat
label_encoder = LabelEncoder()
data['result_encoded'] = label_encoder.fit_transform(data[target])

# Séparer les features et le target
X = data[features]
y = data['result_encoded']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normaliser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Construire le modèle
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes pour W, D, L
])

# Compiler le modèle
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test accuracy: {accuracy * 100:.2f}%')



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Charger les données
data = pd.read_csv('combined_dataset_2021_2022.csv')

# Exclure les matchs de la première semaine
data = data[data['round'] != 'Matchweek 1']

# Prétraitement des données
data['venue'] = data['venue'].apply(lambda x: 1 if x == 'Home' else 0)

# Sélectionner les colonnes pertinentes
features = ['venue', 'moy_goals', 'moy_xg', 'moy_sot']
target = 'result'

# Encoder le résultat
label_encoder = LabelEncoder()
data['result_encoded'] = label_encoder.fit_transform(data[target])

# Séparer les features et le target
X = data[features]
y = data['result_encoded']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normaliser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Construire le modèle
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes pour W, D, L
])

# Compiler le modèle
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test accuracy: {accuracy * 100:.2f}%')

# Tracer les courbes de perte et d'exactitude
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


Ces tracès sont caractéristiques d'un sur-ajustement. L'exactitude de l'entrainement augmente linéaire dans le temps, alors que l'exactitude de la validation se stabilise autour de 95%.
La perte de validation atteint son minimum après 40 Epoch (semblant de bruit), puis décroche que la perte d'entrainement continue à diminuer linéairement.

## 1.3 Augmentation des données par suréchantillonage avec la méthode SMOTE

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Charger les données
data = pd.read_csv('combined_dataset_2021_2022.csv')

# Exclure les matchs de la première semaine
data = data[data['round'] != 'Matchweek 1']

# Prétraitement des données
data['venue'] = data['venue'].apply(lambda x: 1 if x == 'Home' else 0)

# Sélectionner les colonnes pertinentes
features = ['venue', 'moy_goals', 'moy_xg', 'moy_sot']
target = 'result'

# Encoder le résultat
label_encoder = LabelEncoder()
data['result_encoded'] = label_encoder.fit_transform(data[target])

# Séparer les features et le target
X = data[features]
y = data['result_encoded']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Suréchantillonnage avec SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Normaliser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

# Construire le modèle
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes pour W, D, L
])

# Compiler le modèle
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
model.fit(X_train_scaled, y_train_smote, epochs=50, batch_size=32, validation_split=0.2)

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test accuracy: {accuracy * 100:.2f}%')


# 2) Architecture du réseau :
## 2.1 Choix de l'architecture :

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from imblearn.over_sampling import SMOTE

# Charger les données
data = pd.read_csv('combined_dataset_2021_2022.csv')

# Exclure les matchs de la première semaine
data = data[data['round'] != 'Matchweek 1']

# Prétraitement des données
data['venue'] = data['venue'].apply(lambda x: 1 if x == 'Home' else 0)

# Sélectionner les colonnes pertinentes
features = ['venue', 'moy_goals', 'moy_xg', 'moy_sot']
target = 'result'

# Encoder le résultat
label_encoder = LabelEncoder()
data['result_encoded'] = label_encoder.fit_transform(data[target])

# Séparer les features et le target
X = data[features]
y = data['result_encoded']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Suréchantillonnage avec SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Normaliser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

def build_model(architecture_type):
    model = Sequential()
    if architecture_type == 'deeper':
        model.add(Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64, activation='relu'))
    elif architecture_type == 'wider':
        model.add(Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)))
        model.add(Dense(256, activation='relu'))
        model.add(Dense(256, activation='relu'))
    elif architecture_type == 'deep_and_regularized':
        model.add(Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)))
        model.add(Dropout(0.2))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(32, activation='relu'))

    model.add(Dense(3, activation='softmax'))  # 3 classes pour W, D, L
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Tester différentes architectures
for architecture in ['deeper', 'wider', 'deep_and_regularized']:
    model = build_model(architecture)
    print(f"Training model: {architecture}")
    model.fit(X_train_scaled, y_train_smote, epochs=50, batch_size=32, validation_split=0.2)
    loss, accuracy = model.evaluate(X_test_scaled, y_test)
    print(f"Test accuracy for {architecture}: {accuracy * 100:.2f}%")


On compare différentes architectures réseaux : une plus large, une plus profonde et une profonde et régularisée. Il s'avère que c'est cette dernière qui permet d'obtenir la meilleur accuracy, à savoir 48,52%

## 2.2 Choix des hyperparamètres

Maitenant que l'on sait que quel architecture réseau est la meilleure pour notre modèle, on se propose de choisir de manière optimale les hyperparamètres de cette architecture.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

# Charger les données
data = pd.read_csv('combined_dataset_2021_2022.csv')

# Exclure les matchs de la première semaine
data = data[data['round'] != 'Matchweek 1']

# Prétraitement des données
data['venue'] = data['venue'].apply(lambda x: 1 if x == 'Home' else 0)

# Sélectionner les colonnes pertinentes
features = ['venue', 'moy_goals', 'moy_xg', 'moy_sot']
target = 'result'

# Encoder le résultat
label_encoder = LabelEncoder()
data['result_encoded'] = label_encoder.fit_transform(data[target])

# Séparer les features et le target
X = data[features]
y = data['result_encoded']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Suréchantillonnage avec SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Normaliser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

# Définir les hyperparamètres à tester
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [16, 32, 64]
num_epochs = [30, 50, 70]

def build_model(lr):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')  # 3 classes pour W, D, L
    ])
    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Tester différentes combinaisons de hyperparamètres
best_accuracy = 0
best_params = {}
for lr in learning_rates:
    for batch_size in batch_sizes:
        for epochs in num_epochs:
            print(f"Training model with LR={lr}, batch_size={batch_size}, epochs={epochs}")
            model = build_model(lr)
            model.fit(X_train_scaled, y_train_smote, epochs=epochs, batch_size=batch_size, validation_split=0.2)
            loss, accuracy = model.evaluate(X_test_scaled, y_test)
            print(f"Test accuracy: {accuracy * 100:.2f}%")
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {'learning_rate': lr, 'batch_size': batch_size, 'epochs': epochs}

print(f"Best accuracy: {best_accuracy * 100:.2f}% with parameters {best_params}")


Le code ci-dessus nous permet d'obtenir les meilleurs hyperparamètres, et avec une accuracy de 53%, ce qui est une accuracy très proche de celle obtenue seulement avec les côtes, c'est pourquoi on se propose par la suite d'améliorer encore le code avec des techniques de régularisation et de validation.

# 3) Régularisation L2
La régularisation permet de rendre le modèle plus résistant au sur-ajustement. Vérifions si notre modèle est sur-ajusté.
Pour vérifier si le sur-ajustement est présent, les performances des modèles sur des données nouvelles (never-before-seen-data) ont commencé à ralentir, contrairement à leurs performances sur les données de l'ensemble d'entrainement, performances qui s'améliorent toujours au fur et à mesure que l'entrainement progresse.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2
from imblearn.over_sampling import SMOTE

# Charger les données
data = pd.read_csv('combined_dataset_2021_2022.csv')

# Exclure les matchs de la première semaine
data = data[data['round'] != 'Matchweek 1']

# Prétraitement des données
data['venue'] = data['venue'].apply(lambda x: 1 if x == 'Home' else 0)

# Sélectionner les colonnes pertinentes
features = ['venue', 'moy_goals', 'moy_xg', 'moy_sot']
target = 'result'

# Encoder le résultat
label_encoder = LabelEncoder()
data['result_encoded'] = label_encoder.fit_transform(data[target])

# Séparer les features et le target
X = data[features]
y = data['result_encoded']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Suréchantillonnage avec SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Normaliser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

# Définir les hyperparamètres optimaux
best_lr = 0.01
best_batch_size = 16
best_epochs = 30

# Construire le modèle avec régularisation L2
def build_model():
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],), kernel_regularizer=l2(0.0001)),
        Dropout(0.2),
        Dense(128, activation='relu', kernel_regularizer=l2(0.0001)),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=l2(0.0001)),
        Dense(32, activation='relu', kernel_regularizer=l2(0.0001)),
        Dense(3, activation='softmax')  # 3 classes pour W, D, L
    ])
    optimizer = Adam(learning_rate=best_lr)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Créer et entraîner le modèle
model = build_model()
model.fit(X_train_scaled, y_train_smote, epochs=best_epochs, batch_size=best_batch_size, validation_split=0.2)

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")


On obtient une accuracy de 52,59%, donc la technique de régularisation semble confirmer cette approche par les bookmaker d'utiliser ces principales caractéristiques pour prédire le score d'un match

# 4) Techniques de Validation

On va réaliser une première technique de validation croisée K-fold

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import numpy as np

# Charger les données
data = pd.read_csv('combined_dataset_2021_2022.csv')

# Exclure les matchs de la première semaine et réinitialiser l'index
data = data[data['round'] != 'Matchweek 1'].reset_index(drop=True)

# Prétraitement des données
data['venue'] = data['venue'].apply(lambda x: 1 if x == 'Home' else 0)

# Sélectionner les colonnes pertinentes
features = ['venue', 'moy_goals', 'moy_xg', 'moy_sot']
target = 'result'

# Encoder le résultat
label_encoder = LabelEncoder()
data['result_encoded'] = label_encoder.fit_transform(data[target])

# Séparer les features et le target
X = data[features].values
y = data['result_encoded'].values

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Définir les hyperparamètres optimaux
best_lr = 0.01
best_batch_size = 16
best_epochs = 30

# Définir la régularisation L2
reg = l2(0.001)

# Définir la validation croisée K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
accuracies = []

for train_index, test_index in kf.split(X_scaled, y):
    X_train_fold, X_test_fold = X_scaled[train_index], X_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    # Construire le modèle
    model = Sequential([
        Dense(128, activation='relu', kernel_regularizer=reg),
        Dropout(0.2),
        Dense(128, activation='relu', kernel_regularizer=reg),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=reg),
        Dense(32, activation='relu', kernel_regularizer=reg),
        Dense(3, activation='softmax')
    ])

    # Compiler le modèle
    optimizer = Adam(learning_rate=best_lr)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Entraîner le modèle
    print(f'Training for fold {fold_no}...')
    model.fit(X_train_fold, y_train_fold, batch_size=best_batch_size, epochs=best_epochs, verbose=1)

    # Évaluer le modèle
    scores = model.evaluate(X_test_fold, y_test_fold, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[1]} of {scores[1]*100}%')
    accuracies.append(scores[1] * 100)
    fold_no += 1

print(f'Average accuracy over all folds: {np.mean(accuracies)}%')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import numpy as np
import matplotlib.pyplot as plt

# Charger les données
data = pd.read_csv('combined_dataset_2021_2022.csv')

# Exclure les matchs de la première semaine et réinitialiser l'index
data = data[data['round'] != 'Matchweek 1'].reset_index(drop=True)

# Prétraitement des données
data['venue'] = data['venue'].apply(lambda x: 1 if x == 'Home' else 0)

# Sélectionner les colonnes pertinentes
features = ['venue', 'moy_goals', 'moy_xg', 'moy_sot']
target = 'result'

# Encoder le résultat
label_encoder = LabelEncoder()
data['result_encoded'] = label_encoder.fit_transform(data[target])

# Séparer les features et le target
X = data[features].values
y = data['result_encoded'].values

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Définir les hyperparamètres optimaux
best_lr = 0.01
best_batch_size = 16
best_epochs = 30

# Définir la régularisation L2
reg = l2(0.001)

# Définir la validation croisée K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
accuracies = []

for train_index, test_index in kf.split(X_scaled, y):
    X_train_fold, X_test_fold = X_scaled[train_index], X_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    # Créer un ensemble de validation à partir de l'ensemble d'entraînement
    val_split_index = int(0.9 * len(X_train_fold))  # 10% pour la validation
    X_train, X_val = X_train_fold[:val_split_index], X_train_fold[val_split_index:]
    y_train, y_val = y_train_fold[:val_split_index], y_train_fold[val_split_index:]

    # Construire le modèle
    model = Sequential([
        Dense(128, activation='relu', kernel_regularizer=reg),
        Dropout(0.2),
        Dense(128, activation='relu', kernel_regularizer=reg),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=reg),
        Dense(32, activation='relu', kernel_regularizer=reg),
        Dense(3, activation='softmax')
    ])

    # Compiler le modèle
    optimizer = Adam(learning_rate=best_lr)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Entraîner le modèle avec l'ensemble de validation
    print(f'Training for fold {fold_no}...')
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                        batch_size=best_batch_size, epochs=best_epochs, verbose=1)

    # Évaluer le modèle
    scores = model.evaluate(X_test_fold, y_test_fold, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[1]} of {scores[1]*100}%')
    accuracies.append(scores[1] * 100)

    # Tracer les courbes de perte et d'exactitude
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Loss for Fold {fold_no}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Accuracy for Fold {fold_no}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

    fold_no += 1

print(f'Average accuracy over all folds: {np.mean(accuracies)}%')


En utilisant de technique de régularisation encore plus poussée et en ajustant les hyperparamètres du réseau, on obtient une exactitude de prédiction encore meilleure, aux alentours de 48%. Mais il serait difficile d'améliorer ces perfomances en partant de si peu de données. Afin d'améliorer notre exactitude de prédiction sur ce problème, l'étape suivante serait d'utiliser un modèle de machine learning.

La technique de validation semble montrer une baisse d'accuracy, ici de 47,2%, on tend vers un modèle plus proche de la réalite. Réalisons cette même technique de validation en intégrant SMOTE.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from imblearn.over_sampling import SMOTE
import numpy as np

# Charger les données
data = pd.read_csv('combined_dataset_2021_2022.csv')

# Exclure les matchs de la première semaine et réinitialiser l'index
data = data[data['round'] != 'Matchweek 1'].reset_index(drop=True)

# Prétraitement des données
data['venue'] = data['venue'].apply(lambda x: 1 if x == 'Home' else 0)

# Sélectionner les colonnes pertinentes
features = ['venue', 'moy_goals', 'moy_xg', 'moy_sot']
target = 'result'

# Encoder le résultat
label_encoder = LabelEncoder()
data['result_encoded'] = label_encoder.fit_transform(data[target])

# Séparer les features et le target
X = data[features].values
y = data['result_encoded'].values

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Définir les hyperparamètres optimaux
best_lr = 0.01
best_batch_size = 16
best_epochs = 30

# Définir la régularisation L2
reg = l2(0.001)

# Définir la validation croisée K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for train_index, test_index in kf.split(X_scaled, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Diviser les données d'entraînement pour obtenir un ensemble de validation
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Suréchantillonnage avec SMOTE pour les données d'entraînement
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

    # Construire le modèle
    model = Sequential([
        Dense(128, activation='relu', kernel_regularizer=reg),
        Dropout(0.2),
        Dense(128, activation='relu', kernel_regularizer=reg),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=reg),
        Dense(32, activation='relu', kernel_regularizer=reg),
        Dense(3, activation='softmax')
    ])

    # Compiler le modèle
    optimizer = Adam(learning_rate=best_lr)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Entraîner le modèle avec la validation
    model.fit(X_train_smote, y_train_smote, batch_size=best_batch_size, epochs=best_epochs, verbose=1, validation_data=(X_val, y_val))

    # Évaluer le modèle sur l'ensemble de test
    scores = model.evaluate(X_test, y_test, verbose=0)
    accuracies.append(scores[1] * 100)

print(f'Average accuracy over all folds: {np.mean(accuracies)}%')

Réalisons une dernière technique de validation, la validation croisée K-Fold itérative.

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import numpy as np

# Charger les données
data = pd.read_csv('combined_dataset_2021_2022.csv')

# Exclure les matchs de la première semaine et réinitialiser l'index
data = data[data['round'] != 'Matchweek 1'].reset_index(drop=True)

# Prétraitement des données
data['venue'] = data['venue'].apply(lambda x: 1 if x == 'Home' else 0)
features = ['venue', 'moy_goals', 'moy_xg', 'moy_sot']
target = 'result'

# Encoder le résultat
label_encoder = LabelEncoder()
data['result_encoded'] = label_encoder.fit_transform(data[target])

# Séparer les features et le target
X = data[features].values
y = data['result_encoded'].values

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Paramètres de la validation croisée
n_splits = 5
n_repeats = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Paramètres du modèle
best_lr = 0.01
best_batch_size = 16
best_epochs = 30
reg = l2(0.001)

# Boucle pour la validation croisée itérée
overall_scores = []
for i in range(n_repeats):
    fold_scores = []
    for train_index, test_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Construire le modèle
        model = Sequential([
            Dense(128, activation='relu', kernel_regularizer=reg),
            Dropout(0.2),
            Dense(128, activation='relu', kernel_regularizer=reg),
            Dropout(0.2),
            Dense(64, activation='relu', kernel_regularizer=reg),
            Dense(32, activation='relu', kernel_regularizer=reg),
            Dense(3, activation='softmax')
        ])

        # Compiler le modèle
        optimizer = Adam(learning_rate=best_lr)
        model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        # Entraîner le modèle
        model.fit(X_train, y_train, batch_size=best_batch_size, epochs=best_epochs, verbose=0)

        # Évaluer le modèle
        scores = model.evaluate(X_test, y_test, verbose=0)
        fold_scores.append(scores[1] * 100)

    overall_scores.append(np.mean(fold_scores))
    print(f"Iteration {i+1}/{n_repeats}, Mean Accuracy: {np.mean(fold_scores):.2f}%")

print(f'Final Mean Accuracy over all iterations: {np.mean(overall_scores):.2f}%')


In [ ]:
import pandas as pd
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import numpy as np
import matplotlib.pyplot as plt

# Charger les données
data = pd.read_csv('combined_dataset_2021_2022.csv')

# Exclure les matchs de la première semaine et réinitialiser l'index
data = data[data['round'] != 'Matchweek 1'].reset_index(drop=True)

# Prétraitement des données
data['venue'] = data['venue'].apply(lambda x: 1 if x == 'Home' else 0)
features = ['venue', 'moy_goals', 'moy_xg', 'moy_sot']
target = 'result'

# Encoder le résultat
label_encoder = LabelEncoder()
data['result_encoded'] = label_encoder.fit_transform(data[target])

# Séparer les features et le target
X = data[features].values
y = data['result_encoded'].values

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Paramètres de la validation croisée itérative
n_splits = 5
n_repeats = 10
rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)

# Paramètres du modèle
best_lr = 0.01
best_batch_size = 16
best_epochs = 30
reg = l2(0.001)

# Boucle pour la validation croisée itérée
overall_scores = []
fold_histories = []

for i, (train_index, test_index) in enumerate(rkf.split(X_scaled)):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Construire le modèle
    model = Sequential([
        Dense(128, activation='relu', kernel_regularizer=reg),
        Dropout(0.2),
        Dense(128, activation='relu', kernel_regularizer=reg),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=reg),
        Dense(32, activation='relu', kernel_regularizer=reg),
        Dense(3, activation='softmax')
    ])

    # Compiler le modèle
    optimizer = Adam(learning_rate=best_lr)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Entraîner le modèle
    history = model.fit(X_train, y_train, batch_size=best_batch_size, epochs=best_epochs, validation_data=(X_test, y_test), verbose=0)

    # Stocker l'historique pour les tracés
    fold_histories.append(history.history)

    # Évaluer le modèle
    scores = model.evaluate(X_test, y_test, verbose=0)
    overall_scores.append(scores[1] * 100)

    # Tracer les courbes après chaque fold
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Loss for Fold {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Accuracy for Fold {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

print(f'Final Mean Accuracy over all iterations: {np.mean(overall_scores):.2f}%')


# MATRICE DE CORRELATION

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Charger les données
data = pd.read_csv('combined_dataset_2021_2022.csv')

# Transformer la colonne 'result' en valeurs numériques
data['result'] = data['result'].map({'W': 1, 'D': 0, 'L': -1})

# Supprimer les colonnes contenant du texte
data = data.select_dtypes(include=[np.number])  # Garde uniquement les colonnes numériques

# Calculer la matrice de corrélation
correlation_matrix = data.corr()

# Visualiser la matrice de corrélation
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Matrice de Corrélation')
plt.show()
